In [10]:
import os
import json
import random
from PIL import Image, ImageEnhance, ImageOps
from tqdm import tqdm

In [12]:
# Caminhos para as imagens e máscaras processadas
processed_images_dir = '../data/processed/38-Cloud/images/train/'
processed_masks_dir = '../data/processed/38-Cloud/masks/train/'

# Caminhos para as divisões
splits_dir = '../data/processed/38-Cloud/splits/'

# Carregando as divisões do dataset
with open(os.path.join(splits_dir, 'train_files.json'), 'r') as f:
    train_files = json.load(f)
with open(os.path.join(splits_dir, 'val_files.json'), 'r') as f:
    val_files = json.load(f)


print(f"Número de imagens de treinamento: {len(train_files)}")
print(f"Número de imagens de validação: {len(val_files)}")

Número de imagens de treinamento: 23670
Número de imagens de validação: 2631


Funções de Transformação

In [13]:
def random_horizontal_flip(image, mask, p=0.5):
    if random.random() < p:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
        mask = mask.transpose(Image.FLIP_LEFT_RIGHT)
    return image, mask

def random_vertical_flip(image, mask, p=0.5):
    if random.random() < p:
        image = image.transpose(Image.FLIP_TOP_BOTTOM)
        mask = mask.transpose(Image.FLIP_TOP_BOTTOM)
    return image, mask

def random_rotation(image, mask, degrees=15):
    angle = random.uniform(-degrees, degrees)
    image = image.rotate(angle, resample=Image.BILINEAR)
    mask = mask.rotate(angle, resample=Image.NEAREST)
    return image, mask

def random_zoom(image, mask, scale_range=(0.9, 1.1)):
    scale = random.uniform(*scale_range)
    w, h = image.size
    new_w, new_h = int(w * scale), int(h * scale)
    image = image.resize((new_w, new_h), resample=Image.BILINEAR)
    mask = mask.resize((new_w, new_h), resample=Image.NEAREST)

    # Crop or pad to original size
    if scale < 1.0:
        # Pad
        pad_w = (w - new_w) // 2
        pad_h = (h - new_h) // 2
        image = ImageOps.expand(image, border=(pad_w, pad_h), fill=0)
        mask = ImageOps.expand(mask, border=(pad_w, pad_h), fill=0)
        image = image.crop((0, 0, w, h))
        mask = mask.crop((0, 0, w, h))
    else:
        # Crop
        left = (new_w - w) // 2
        top = (new_h - h) // 2
        image = image.crop((left, top, left + w, top + h))
        mask = mask.crop((left, top, left + w, top + h))

    return image, mask

def random_brightness_contrast(image, brightness_range=(0.8, 1.2), contrast_range=(0.8, 1.2)):
    brightness_factor = random.uniform(*brightness_range)
    contrast_factor = random.uniform(*contrast_range)

    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness_factor)

    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast_factor)

    return image

Aplicando as Transformações e Salvando os Dados Aumentados

In [14]:
# Número de aumentos por imagem
augmentations_per_image = 2 # originais + 2 * aumentadas

# Diretórios para salvar as imagens e máscaras aumentadas
augmented_images_dir = '../data/processed/38-Cloud/images/train_augmented/'
augmented_masks_dir = '../data/processed/38-Cloud/masks/train_augmented/'

os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_masks_dir, exist_ok=True)

Loop de Aumento de Dados

In [15]:
augmented_train_files = []

for img_file in tqdm(train_files, desc="Aplicando aumento de dados"):
    img_path = os.path.join(processed_images_dir, img_file)
    mask_path = os.path.join(processed_masks_dir, img_file)  # Assumindo que a máscara tem o mesmo nome

    # Carregar imagem e máscara
    image = Image.open(img_path).convert('RGB')
    mask = Image.open(mask_path).convert('L')

    for i in range(augmentations_per_image):
        augmented_image = image.copy()
        augmented_mask = mask.copy()

        # Aplicar as transformações
        augmented_image, augmented_mask = random_horizontal_flip(augmented_image, augmented_mask, p=0.5)
        augmented_image, augmented_mask = random_vertical_flip(augmented_image, augmented_mask, p=0.5)
        augmented_image, augmented_mask = random_rotation(augmented_image, augmented_mask, degrees=15)
        augmented_image, augmented_mask = random_zoom(augmented_image, augmented_mask, scale_range=(0.9, 1.1))
        augmented_image = random_brightness_contrast(augmented_image, brightness_range=(0.8, 1.2), contrast_range=(0.8, 1.2))

        # Salvar a imagem e máscara aumentadas
        base_name = os.path.splitext(img_file)[0]
        augmented_img_file = f"{base_name}_aug_{i}.png"
        augmented_mask_file = f"{base_name}_aug_{i}.png"

        augmented_image.save(os.path.join(augmented_images_dir, augmented_img_file))
        augmented_mask.save(os.path.join(augmented_masks_dir, augmented_mask_file))

        augmented_train_files.append(augmented_img_file)

# Combinar os arquivos originais e aumentados
train_files_augmented = train_files + augmented_train_files

print(f"Número original de imagens de treinamento: {len(train_files)}")
print(f"Número de imagens aumentadas: {len(augmented_train_files)}")
print(f"Total de imagens de treinamento após aumento: {len(train_files_augmented)}")

Aplicando aumento de dados: 100%|██████████| 23670/23670 [1:01:23<00:00,  6.43it/s]

Número original de imagens de treinamento: 23670
Número de imagens aumentadas: 47340
Total de imagens de treinamento após aumento: 71010


Atualizando as Divisões do Dataset

In [16]:
# Atualizar o train_files.json
with open(os.path.join(splits_dir, 'train_files_augmented.json'), 'w') as f:
    json.dump(train_files_augmented, f)